In [1]:
from baybe.targets import NumericalTarget
from baybe.objectives import SingleTargetObjective
from baybe.parameters import CustomDiscreteParameter
from baybe.searchspace import SearchSpace
from baybe import Campaign
from baybe.exceptions import NotEnoughPointsLeftError
import pandas as pd
import json, math
import random
import numpy as np
import torch

In [2]:
file_path = f"../../../data/data_61/data_BO_61_CO_biphenyl/data_Reaction_CO_biphenyl.csv"
df = pd.read_csv(file_path)
cols = ["HOMO","LUMO","E_S1","f_S1","E_T1","dEST","dDM"]
df_data = df.set_index("ID")[cols].copy()

# ID -> Yield
def evaluate(ID):
    y = df.set_index("ID")["yield"].get(ID)
    return y

# ID -> index
def convert(ID):
    idx = df.index[df["ID"].eq(ID)].item()
    return idx

target = NumericalTarget(name="Yield")
objective = SingleTargetObjective(target=target)

parameters = [
        CustomDiscreteParameter(
            name="ID",
            data=df_data,
    ),
]

searchspace = SearchSpace.from_product(parameters)

rows = []
for row_idx in range(60):
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    
    idx_list, yield_list = [], []
    campaign = Campaign(searchspace, objective)
    
    ci = df.columns.get_loc("ID")  
    first_id = df.iat[row_idx, ci]
    y = evaluate(first_id)
    idx_list.append(convert(first_id))
    yield_list.append(int(y))
    df_ini = pd.DataFrame({
        "ID": [first_id],
        "Yield":[float(y)]
    })
    campaign.add_measurements(df_ini)
    
    while True:
        try:
            df_rec = campaign.recommend(batch_size=1) 
        except NotEnoughPointsLeftError:
            break
    
        y = evaluate(df_rec.iat[0, 0])
    
        df_rec["Yield"] = [float(y)]
        idx_list.append(convert(df_rec.iat[0, 0]))
        yield_list.append(int(y))
        campaign.add_measurements(df_rec)
        if convert(df_rec.iat[0, 0]) == 60:
            break
    rows.append({"selected_indices_per_step": idx_list, "actual_rewards_per_step": yield_list})
    
df_out = pd.DataFrame.from_records(rows)   
df_out.to_csv(f"../../results_OoS/BOsearch_DFT/BOsearch_Reaction_CO_biphenyl_result.csv", index=False)